In [19]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/PURI/')
import torch

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [21]:
class FiveLayerResNet(nn.Module):
    def __init__(self, num_classes=7):
        super(FiveLayerResNet, self).__init__()
        SQ1, SQ2, SQ3, SQ4, SQ5 = 32, 64, 128, 256, 512

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=SQ1, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(SQ1)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv2 = nn.Conv2d(in_channels=SQ1, out_channels=SQ2, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(SQ2)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.skip2 = nn.Sequential(
            nn.Conv2d(SQ1, SQ2, kernel_size=1, bias=False),
            nn.BatchNorm2d(SQ2)
        )

        self.conv3 = nn.Conv2d(in_channels=SQ2, out_channels=SQ3, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(SQ3)
        self.skip3 = nn.Sequential(
            nn.Conv2d(SQ2, SQ3, kernel_size=1, bias=False),
            nn.BatchNorm2d(SQ3)
        )

        self.conv4 = nn.Conv2d(in_channels=SQ3, out_channels=SQ4, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(SQ4)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.skip4 = nn.Sequential(
            nn.Conv2d(SQ3, SQ4, kernel_size=1, bias=False),
            nn.BatchNorm2d(SQ4)
        )

        self.conv5 = nn.Conv2d(in_channels=SQ4, out_channels=SQ5, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(SQ5)
        self.skip5 = nn.Sequential(
            nn.Conv2d(SQ4, SQ5, kernel_size=1, bias=False),
            nn.BatchNorm2d(SQ5)
        )

        whole_size_cons = SQ5 * 6 * 6  # 512 * 6 * 6
        self.fc1 = nn.Linear(whole_size_cons, SQ5)
        self.fc2 = nn.Linear(SQ5, SQ3)
        self.fc3 = nn.Linear(SQ3, num_classes)

    def forward(self, x):
        #  1
        kv1 = self.conv1(x)
        kv1_bn = self.bn1(kv1)
        kv1_relu = F.relu(kv1_bn)
        x = self.pool1(kv1_relu)

        #  2 with skip
        identity2 = self.skip2(x)
        kv2 = self.conv2(x)
        kv2_bn = self.bn2(kv2)
        kv2_relu = F.relu(kv2_bn)
        kv2_pooled = self.pool2(kv2_relu)
        identity2_pooled = self.pool2(identity2)
        x = kv2_pooled + identity2_pooled
        x = F.relu(x)

        #  3 with skip
        identity3 = self.skip3(x)
        kv3 = self.conv3(x)
        kv3_bn = self.bn3(kv3)
        kv3_relu = F.relu(kv3_bn)
        x = kv3_relu + identity3
        x = F.relu(x)

        #  4 with skip
        identity4 = self.skip4(x)
        kv4 = self.conv4(x)
        kv4_bn = self.bn4(kv4)
        kv4_relu = F.relu(kv4_bn)
        kv4_pooled = self.pool4(kv4_relu)
        identity4_pooled = self.pool4(identity4)
        x = kv4_pooled + identity4_pooled
        x = F.relu(x)

        #  5 with skip
        identity5 = self.skip5(x)
        kv5 = self.conv5(x)
        kv5_bn = self.bn5(kv5)
        kv5_relu = F.relu(kv5_bn)
        x = kv5_relu + identity5
        x = F.relu(x)
        inp_br_x = x.size(0)
        x = x.view(inp_br_x, -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [22]:

model=None
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

save_path = '/content/drive/MyDrive/PURI/my_models/my_best_model.pth'

if torch.cuda.is_available():
    model = torch.load(save_path,weights_only=False)
else:
    model = torch.load(save_path, map_location=device, weights_only=False)


model.to(device)
model.eval()
print("Model loaded successfully")


Using device: cuda
Model loaded successfully


In [23]:
type(model)

my_models.five_layer_batchnorm_skipping.FiveLayerResNet

In [24]:

import sys
import os
sys.path.append('/content/drive/MyDrive/PURI/data')
try:
    from my_data_folder.my_transformations import VAL_TRANSFORM
    from my_data_folder.my_data import load_my_data, FER2013Dataset
    print(" Imports worked")
except ImportError as e:
    print(f"Import failed: {e}")

 Imports worked


In [25]:
train_df, val_df, test_df = load_my_data()

if test_df is not None:
    print(f"Test data loaded")
    print(f"Test dataset size: {len(test_df)}")
    print(f"Emotions in test set:")
    print(test_df['emotion'].value_counts().sort_index())
else:
    print(" Could not load test data")

 Using FER2013 from MyDrive
Train : 28709
Validation : 3589
Test : 3589
Test data loaded
Test dataset size: 3589
Emotions in test set:
emotion
0    467
1     56
2    496
3    895
4    653
5    415
6    607
Name: count, dtype: int64


In [26]:
from torch.utils.data import DataLoader


test_dataset = FER2013Dataset(test_df, transform=VAL_TRANSFORM)


test_loader = DataLoader(
    test_dataset,
    batch_size=100,
    shuffle=False,
    num_workers=2
)

print(f"Test loader created successfully")
print(f"Number of batches: {len(test_loader)}")
print(f"Total test samples: {len(test_dataset)}")
print(f"Batch size: 100")

Test loader created successfully
Number of batches: 36
Total test samples: 3589
Batch size: 100


In [27]:
import torch.nn.functional as F


In [28]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

print("Testing my best model")
print(" ")

correct = 0
total = 0
class_correct = list(0. for i in range(7))
class_total = list(0. for i in range(7))

model.eval()

with torch.no_grad():
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = data.to(device), target.to(device)

        outputs = model(data)
        _, predicted = torch.max(outputs, 1)

        total += target.size(0)
        correct += (predicted == target).sum().item()
        c = (predicted == target).squeeze()
        for i in range(target.size(0)):
            label = target[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

        if batch_idx % 5 == 0:
            print(f'Processing batch {batch_idx + 1}/{len(test_loader)}... Current accuracy: {100 * correct / total:.1f}%')


print(f'Final res:')
print(" ")
print(f'Overall Test Accuracy: {100 * correct / total:.2f}%')
print(f'Correct predictions: {correct}/{total}')

print(f'Before Trans: ')
print(" ")
for i in range(7):
    if class_total[i] > 0:
        accuracy = 100 * class_correct[i] / class_total[i]
        print(f'{emotion_labels[i]:>8}: {accuracy:5.1f}% ({int(class_correct[i]):>3}/{int(class_total[i]):>3})')

print(f'summary')
print("" )
print(f'Training Accuracy:   98.3%')
print(f'Validation Accuracy: 62.9%')
print(f'Test Accuracy:       {100 * correct / total:.2f}%')

test_acc = 100 * correct / total
val_acc = 62.9
if test_acc > val_acc + 2:
    print(f'test > Val: Model generalizes well!')
elif test_acc < val_acc - 5:
    print(f'Test < Val: Possible overfitting detected')
else:
    print(f'Test ≈ Val: Good generalization')

Testing my best model
 
Processing batch 1/36... Current accuracy: 62.0%
Processing batch 6/36... Current accuracy: 61.5%
Processing batch 11/36... Current accuracy: 60.5%
Processing batch 16/36... Current accuracy: 60.8%
Processing batch 21/36... Current accuracy: 60.3%
Processing batch 26/36... Current accuracy: 60.9%
Processing batch 31/36... Current accuracy: 60.8%
Processing batch 36/36... Current accuracy: 60.6%
Final res:
 
Overall Test Accuracy: 60.60%
Correct predictions: 2175/3589
Before Trans: 
 
   Angry:  45.0% (210/467)
 Disgust:  67.9% ( 38/ 56)
    Fear:  39.5% (196/496)
   Happy:  80.7% (722/895)
     Sad:  50.8% (332/653)
Surprise:  71.8% (298/415)
 Neutral:  62.4% (379/607)
summary

Training Accuracy:   98.3%
Validation Accuracy: 62.9%
Test Accuracy:       60.60%
Test ≈ Val: Good generalization
